In [ ]:
!pip install beautifulsoup4 openpyxl requests pandas feedparser openai

In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import openai
import re

# 기본 URL
base_url = "https://news.daum.net/breakingnews/digital?page="

# 오늘 날짜 가져오기
today = (datetime.today() - timedelta(days=1)).strftime('%Y%m%d')
#today = datetime.today().strftime('%Y%m%d') 12시에 날짜가 바껴 뉴스 사라짐

# 비어있는 DataFrame 생성
all_data = pd.DataFrame(columns=["Title", "Link"])

# 마지막 페이지 모르기에 첫 페이지에서 시작
response = requests.get(base_url + "1&regDate=" + today)
response.raise_for_status()  # 응답코드 200 아니면 오류 발생
soup = BeautifulSoup(response.text, 'html.parser')

# <span> tag 내부의 page 숫자들 추출
page_numbers = [int(tag.text) for tag in soup.select('.inner_paging .num_page') if tag.text.isdigit()]

# 마지막 페이지 찾기
last_page_number = max(page_numbers) if page_numbers else 1

# 페이지 돌면서 url 추출
for page in range(1, last_page_number + 1):
    # 페이지 별 오늘 날짜 뉴스 추출
    url = base_url + str(page) + "&regDate=" + today

    # 내용 가져오기
    response = requests.get(url)
    response.raise_for_status()  
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # news item 추출
    news_items = soup.select('.box_etc li > div.cont_thumb > strong.tit_thumb')

    # 제목, 링크 추출 후 df 추가
    for item in news_items:
        title = item.select_one('a.link_txt').text
        link = item.select_one('a.link_txt')['href']
        all_data = all_data.append({"Title": title, "Link": link}, ignore_index=True)

# 뉴스 기사 본문 추출
def extract_article_content(url):
    with session.get(url) as response:
        soup = BeautifulSoup(response.text, 'html.parser')
        # <section> tag 내부 <p> tag 추출
        article_paragraphs = soup.select("div.article_view section p")
        article_text = '\n'.join([para.text for para in article_paragraphs])
        return article_text if article_text else ""

# url 링크 리스트만들기
links_to_scrape = all_data["Link"].tolist()

# requests요청 위한 session 사용
with requests.Session() as session:
    # requests 병렬 처리
    with ThreadPoolExecutor(max_workers=10) as executor:
        # 병렬 처리로 인해 순서가 다를 수 있으니 순서가 유지되도록 보장
        all_data["article"] = list(executor.map(extract_article_content, links_to_scrape))

all_data.to_excel('news.xlsx')


# OpenAI API 키 설정
openai.api_key = ''

# 엑셀 파일 불러오기
#file_path = "news.xlsx"
#data = pd.read_excel(file_path)
data = all_data

# 중요 뉴스 가져오기
def get_important_titles_for_chunk(chunk):
    titles = "\n".join(chunk)
    prompt_text = f"Given the following news titles, select the top 3 most important ones:\n{titles}\n\nTop 3 important news titles are:"
    response = openai.Completion.create(
        model="text-davinci-003", # 여긴왜 3.5버전 안썼지?
        prompt=prompt_text,
        max_tokens=200
    )
    important_titles = response.choices[0].text.strip().split("\n")
    return [re.sub(r"^\d+\.\s*", "", title) for title in important_titles]

# 10개씩 청크로 나누기
chunk_size = 10
chunks = [data['Title'][i:i+chunk_size].tolist() for i in range(0, len(data), chunk_size)]

all_important_titles = []
for chunk in chunks:
    all_important_titles.extend(get_important_titles_for_chunk(chunk))

# 최종적으로 원하는 숫자만큼 중요 기사 선택 (예: 상위 5개)
final_important_titles = all_important_titles[:5]

# 중요한 뉴스 제목에 해당하는 행들을 새로운 DataFrame으로 가져오기
important_news_df = data[data['Title'].isin(final_important_titles)]

print(important_news_df)

# 뉴스 요약
def summarize_news(news_body):

    # 뉴스 본문을 요약하려면 사용자의 지시에 따라 대화를 정의
    conversation = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"주어진 기사를 짧게 요약해줘 한글로: {news_body}"}
    ]

    # 최대 토큰 개수 지정
    max_tokens = 100

    # api 요청
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=conversation,
    )
    # 요약 추출
    summary = response["choices"][0]["message"]["content"]
    return summary

def main(news_body):
    # 뉴스 요약
    summary = summarize_news(news_body)
    print("Summary:")
    print(summary)
    return summary

# 뉴스 요약
summary_list = important_news_df['article'].apply(main)
important_news_df['summary']=summary_list

def extract_keywords_from_content(content):
    # 본문의 시작 부분과 끝 부분을 결합
    truncated_content = content[:600] + "\n...\n" + content[-600:]

    prompt_text = f"주어진 뉴스 기사의 본문을 보고 핵심 키워드를 `핵심 키워드 : 축구, 이강인, 멘체스터 유나이티드' 와 같은 형태로 추출해줘. ###{truncated_content}###"
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt_text,
        max_tokens=200  # 조절 가능
    )
    keywords = response.choices[0].text.strip().split(", ")
    keywords[0]=keywords[0].replace("핵심 키워드","")
    keywords[0]=keywords[0].replace(":","").strip()
    return keywords

# 각 본문에 대해 주요 키워드를 추출
keywords_list = important_news_df['article'].apply(extract_keywords_from_content)

# 새로운 DataFrame 생성
keywords_df = important_news_df.copy()

keywords_df['keywords']=keywords_list
print(keywords_df)

# 엑셀 파일로 저장
file_name = "gpt_news.xlsx"
keywords_df.to_excel(file_name, index=False)

# 데이터프레임의 내용을 HTML 형식으로 변환
email_content = keywords_df.to_html()

# DataFrame을 원하는 형태로 리스트로 변환
html_content = ""
for index, row in keywords_df.iterrows():
    title = row["Title"]
    link = row["Link"]
    summary = row["summary"]
    keyword = row["keywords"]

    # 하이퍼링크 추가
    title_with_link = f'<a href="{link}">{title}</a>'

    # 형식에 맞게 포맷팅
    formatted_item = f"<h2>제목 : {title_with_link}</h2><br>요약<br>{summary}<br>키워드: {keyword[:5]}<br>"

    html_content += formatted_item

# 이메일 설정
sender_email = ""
receiver_email = ""
password = ""
subject = "Today's Top 3 Important News Titles"

message = MIMEMultipart("alternative")
message["Subject"] = subject
message["From"] = sender_email
message["To"] = receiver_email
part = MIMEText(email_content, "html")
message.attach(part)

# 이메일 보내기
with smtplib.SMTP_SSL("smtp.naver.com", 465) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message.as_string())


C:\Users\hwang\AppData\Local\Temp\ipykernel_12680\1948430681.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append({"Title": title, "Link": link}, ignore_index=True)
C:\Users\hwang\AppData\Local\Temp\ipykernel_12680\1948430681.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append({"Title": title, "Link": link}, ignore_index=True)
C:\Users\hwang\AppData\Local\Temp\ipykernel_12680\1948430681.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append({"Title": title, "Link": link}, ignore_index=True)
C:\Users\hwang\AppData\Local\Temp\ipykernel_12680\1948430681.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a

                                               Title  \
3                               '로아사랑단' 카멘 레이드 최초 정복   
8                          "방심위 인터넷 언론 콘텐츠 심의는 위법행위”   
10         “이러면 차라리 안봐?” 역대급 터지자, 월 4천원 인상…‘탄식’ 쏟아진다   
12  [TGS 2023] 세가, '용과같이'-'페르소나'-'소닉' 신작으로 TGS를 지배하다   

                                      Link  \
3   https://v.daum.net/v/20230923215406485   
8   https://v.daum.net/v/20230923190728038   
10  https://v.daum.net/v/20230923185107862   
12  https://v.daum.net/v/20230923183603650   

                                              article  
3   \n스마일게이트 MMORPG '로스트아크' 최강 레이드 카멘 하드 퍼스트 클리어 공...  
8   (지디넷코리아=박수형 기자)방송통신심의원회가 인터넷 언론사의 온라인 콘텐츠에 대해 ...  
10  \n[헤럴드경제= 박영훈 기자] “무빙 빼면 볼게 없는데, 이용료 올리면 안보는게 ...  
12  엄청난 인파에 발디딜 틈 조차 없었다. 세가와 아틀라스 합동 부스는 TGS 2023...  
Summary:
스마일게이트 MMORPG '로스트아크'의 최강 레이드 카멘 하드 퍼스트 클리어 공격대가 출시 11일 만에 등장했습니다. 로아사랑단 공격대가 하드 난이도 3관문 최초 공략에 성공하여, 어둠군단장 카멘 레이드의 최강이라고 불리는 위용을 증명했습니다. 로아사랑단 공격대는 8인으로 구성되어 13일부터 입장 가능하고, 최고 수준의 레벨과 난이도를 가지고 있습니다.

C:\Users\hwang\AppData\Local\Temp\ipykernel_12680\1948430681.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  important_news_df['summary']=summary_list


                                               Title  \
3                               '로아사랑단' 카멘 레이드 최초 정복   
8                          "방심위 인터넷 언론 콘텐츠 심의는 위법행위”   
10         “이러면 차라리 안봐?” 역대급 터지자, 월 4천원 인상…‘탄식’ 쏟아진다   
12  [TGS 2023] 세가, '용과같이'-'페르소나'-'소닉' 신작으로 TGS를 지배하다   

                                      Link  \
3   https://v.daum.net/v/20230923215406485   
8   https://v.daum.net/v/20230923190728038   
10  https://v.daum.net/v/20230923185107862   
12  https://v.daum.net/v/20230923183603650   

                                              article  \
3   \n스마일게이트 MMORPG '로스트아크' 최강 레이드 카멘 하드 퍼스트 클리어 공...   
8   (지디넷코리아=박수형 기자)방송통신심의원회가 인터넷 언론사의 온라인 콘텐츠에 대해 ...   
10  \n[헤럴드경제= 박영훈 기자] “무빙 빼면 볼게 없는데, 이용료 올리면 안보는게 ...   
12  엄청난 인파에 발디딜 틈 조차 없었다. 세가와 아틀라스 합동 부스는 TGS 2023...   

                                              summary  \
3   스마일게이트 MMORPG '로스트아크'의 최강 레이드 카멘 하드 퍼스트 클리어 공격...   
8   방송통신심의원회가 인터넷 언론의 콘텐츠 심의에 대해 위법한 심의라는 지적이 제기되었...   
10  디즈니+의 인기 작품인 '무빙'이 끝나면서 이용자

In [ ]:
# 100초 정도 걸림